In [1]:
import pymssql
import _mssql
import datetime
from decimal import Decimal

In [2]:
pymssql.__version__

'2.2.0'

In [3]:
server = 'h9ggwlagd1.database.windows.net'
user = 'modelread'
password = '4fSHRXwd4u'

In [4]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['tmldb']

In [5]:
def convert_dates(values):
    k,v = values
    if type(v) == datetime.date:
        return k, datetime.datetime.combine(
                v,
                datetime.datetime.min.time())
    if type(v) == Decimal:
        return k, float(v)
    else:
        return k, v
    
d = {'a': datetime.date.min, 'b': 'adadad'}

In [6]:
conn = pymssql.connect(server, user+"@"+server, password, "TMLDB")

In [8]:
c2 = conn.cursor(as_dict=True)

In [15]:
colname = 'contracts'
c2.execute('SELECT * FROM cqgdb.tbl'+colname)

In [16]:
collection = db[colname]

In [17]:
cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))


Done 6050 rows


In [11]:
pymongo.__version__

'3.2.2'

In [ ]:
c2.rowcount

In [18]:
import sys
sys.path.append('/home/ubertrader/Dropbox/tmqrexo/tmqrv')
sys.path.append('/home/ubertrader/Dropbox/tmqrexo/tmqrv/exobuilder/tests')

In [22]:
from exobuilder.contracts.futureschain import FuturesChain
from exobuilder.contracts.futurecontract import FutureContract
from exobuilder.tests.assetindexdict import AssetIndexDicts
from datetime import datetime, date
from exobuilder.contracts.instrument import Instrument
from exobuilder.data.datasource_mongo import DataSourceMongo
from exobuilder.data.assetindex_mongo import AssetIndexMongo

In [21]:
assetindex = AssetIndexDicts()
symbol = 'EP'
date = datetime(2014, 1, 6, 10, 15, 0)
futures_limit = 3
instrument = Instrument(assetindex, symbol, date, futures_limit, options_limit=10)
fut_chain = FuturesChain(instrument)

AttributeError: 'AssetIndexDicts' object has no attribute 'assetindex'

In [23]:
mongo_connstr = 'mongodb://localhost:27017/'
mongo_db_name = 'tmldb'
assetindex = AssetIndexMongo(mongo_connstr, mongo_db_name)
date = datetime(2014, 1, 6, 10, 15, 0)
futures_limit = 3
options_limit = 10

datasource = DataSourceMongo(mongo_connstr, mongo_db_name, assetindex, date, futures_limit, options_limit)

In [10]:
fut_id = 724 #datasource['ES'].futures[0].dbid

In [ ]:
c2 = conn.cursor(as_dict=True)
c2.execute("SELECT * FROM cqgdb.tblbardata where idcontract = '{0}'".format(fut_id))
collection = db['futures_data']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
        print(data)
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))

{'_id': ObjectId('5798d1699125441f1b8059d6'), 'high': 1327.75, 'low': 1327.0, 'idcontract': 724, 'datetime': datetime.datetime(2011, 2, 11, 13, 14), 'close': 1327.25, 'open': 1327.5, 'idbardata': 19394387, 'volume': 13226.0}
{'_id': ObjectId('5798d1699125441f1b8059d7'), 'high': 1326.75, 'low': 1326.0, 'idcontract': 724, 'datetime': datetime.datetime(2011, 2, 13, 15, 0), 'close': 1326.25, 'open': 1326.0, 'idbardata': 19394388, 'volume': 1542.0}
{'_id': ObjectId('5798d1699125441f1b8059d8'), 'high': 1326.5, 'low': 1326.0, 'idcontract': 724, 'datetime': datetime.datetime(2011, 2, 13, 15, 1), 'close': 1326.25, 'open': 1326.5, 'idbardata': 19394389, 'volume': 128.0}
{'_id': ObjectId('5798d1699125441f1b8059d9'), 'high': 1326.25, 'low': 1326.0, 'idcontract': 724, 'datetime': datetime.datetime(2011, 2, 13, 15, 2), 'close': 1326.25, 'open': 1326.25, 'idbardata': 19394390, 'volume': 56.0}
{'_id': ObjectId('5798d1699125441f1b8059da'), 'high': 1326.5, 'low': 1326.25, 'idcontract': 724, 'datetime': 

In [11]:
# Real price on ("2014-01-06 10:15:00")
instrument.futures[0]._price = 1819.5

In [26]:
opt_chain = datasource['ES'].futures[0].options[0]

In [27]:
options_dbids = []
for optpair in opt_chain:
    options_dbids.append(optpair.C.dbid)
    options_dbids.append(optpair.P.dbid)

In [10]:
options_dbids = [195035,]

In [11]:
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tbloptiondata where idoption in {0}".format(tuple(options_dbids))
print(query)
c2.execute(query)
collection = db['options_data']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))

SELECT * FROM cqgdb.tbloptiondata where idoption in (195035,)


ProgrammingError: (102, b"Incorrect syntax near ')'.DB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\n")

In [24]:
db_id = 195035
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tbloptions where idoption = {0}".format(db_id)
print(query)
c2.execute(query)
collection = db['options_data']

cnt = 0
for row in c2:
    print(row)

print(row)

SELECT * FROM cqgdb.tbloptions where idoption = 195035
{'idcontract': 741, 'year': 2011, 'cqgsymbol': 'F.TYAH11', 'expirationdate': datetime.date(2011, 3, 22), 'monthint': 3, 'contractname': 'F.US.TYAH11', 'idinstrument': 2, 'month': 'H'}


In [23]:
db_id = 741
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tblcontracts where idcontract = {0}".format(db_id)
print(query)
c2.execute(query)
collection = db['options_data']

cnt = 0
for row in c2:
    print(row)

print(row)

SELECT * FROM cqgdb.tblcontracts where idcontract = 741
{'idcontract': 741, 'year': 2011, 'cqgsymbol': 'F.TYAH11', 'expirationdate': datetime.date(2011, 3, 22), 'monthint': 3, 'contractname': 'F.US.TYAH11', 'idinstrument': 2, 'month': 'H'}
{'idcontract': 741, 'year': 2011, 'cqgsymbol': 'F.TYAH11', 'expirationdate': datetime.date(2011, 3, 22), 'monthint': 3, 'contractname': 'F.US.TYAH11', 'idinstrument': 2, 'month': 'H'}


In [26]:
db_id = 2
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tblinstruments where idinstrument = {0}".format(db_id)
print(query)
c2.execute(query)
collection = db['options_data']

cnt = 0
for row in c2:
    print(row)


SELECT * FROM cqgdb.tblinstruments where idinstrument = 2
{'spantickdisplay': 5.0, 'optionticksize': 0.015625, 'ticksize': 0.015625, 'optionstrikedisplay': 25.0, 'tickvalue': 15.625, 'admoptionftpfilestrikedisplay': 2.5, 'listedspread': 0, 'spanoptioncode': '21', 'usedailycustomdata': 1, 'exchangesymbol': 'ZN', 'spanoptionticksize': 0.015625, 'customdayboundarytime': datetime.datetime(1900, 1, 1, 11, 5), 'spanoptionstart': datetime.date(2008, 1, 2), 'optionadmstrikedisplay': 2.5, 'instrumentid_eod': 0, 'optiontickdisplay': 1.0, 'decisionoffsetminutes': 5, 'cqgsymbol': 'TYA', 'admcode': '21', 'notes': '', 'admfuturepricefactor': 1.0, 'datastart': datetime.date(2008, 8, 10), 'secondaryoptiontickdisplay': 0.0, 'optionexchangesymbolTT': 'OZN', 'admoptionpricefactor': 1.0, 'enabled': 1, 'stoptype': 1, 'timeshifthours': 0, 'idAssetClass': 1, 'limittickoffset': 10, 'description': 'US 10 YR Bond', 'secondaryoptiontickvalue': 0.0, 'optionstrikedisplayTT': 0.25, 'margin': 1623, 'spanstrikedispla

## RFR

In [30]:
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tbloptioninputdata"
print(query)
c2.execute(query)

collection = db['options_data_inputs']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break
print('Done {0} rows'.format(cnt))

SELECT * FROM cqgdb.tbloptioninputdata
Done 3086 rows
